<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [6]:
import folium
import pandas as pd
from IPython.display import display, HTML

# Launch site locations
launch_sites = pd.DataFrame({
    'LaunchSite': [
        'CCAFS LC-40',
        'CCAFS SLC-40',
        'KSC LC-39A',
        'VAFB SLC-4E'
    ],
    'Lat': [
        28.5618571,
        28.5618571,
        28.6080585,
        34.632834
    ],
    'Long': [
        -80.577366,
        -80.577366,
        -80.604444,
        -120.610746
    ]
})

# Create map
spacex_map = folium.Map(
    location=[launch_sites['Lat'][0], launch_sites['Long'][0]],
    zoom_start=5
)

# Add markers
for _, site in launch_sites.iterrows():
    folium.Marker(
        location=[site['Lat'], site['Long']],
        popup=site['LaunchSite'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(spacex_map)

# FORCE display (bypass trust issue)
display(HTML(spacex_map._repr_html_()))


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [7]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [8]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [9]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [11]:
import folium
from folium.features import DivIcon
from IPython.display import display, HTML

# NASA Johnson Space Center coordinates
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Create map
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Create a circle at NASA JSC
circle = folium.Circle(
    nasa_coordinate,
    radius=1000,
    color='#d35400',
    fill=True
).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a text marker using DivIcon
marker = folium.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size:12px; color:#d35400;"><b>NASA JSC</b></div>'
    )
)

# Add to map
site_map.add_child(circle)
site_map.add_child(marker)

# FORCE display (important in restricted labs)
display(HTML(site_map._repr_html_()))


and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [13]:
import folium
import pandas as pd
from folium.features import DivIcon
from IPython.display import display, HTML

# Launch site dataframe
launch_sites = pd.DataFrame({
    'LaunchSite': [
        'CCAFS LC-40',
        'CCAFS SLC-40',
        'KSC LC-39A',
        'VAFB SLC-4E'
    ],
    'Lat': [
        28.5618571,
        28.5618571,
        28.6080585,
        34.632834
    ],
    'Long': [
        -80.577366,
        -80.577366,
        -80.604444,
        -120.610746
    ]
})

# NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Create map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add circle and marker for each launch site
for _, site in launch_sites.iterrows():
    coordinate = [site['Lat'], site['Long']]
    
    # Circle
    folium.Circle(
        coordinate,
        radius=1000,
        color='#000000',
        fill=True
    ).add_child(folium.Popup(site['LaunchSite'])).add_to(site_map)
    
    # Text marker
    folium.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<div style="font-size:12px; color:#d35400;"><b>%s</b></div>' 
                 % site['LaunchSite']
        )
    ).add_to(site_map)

# Force display (important for restricted labs)
display(HTML(site_map._repr_html_()))


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [14]:
# Task 2: Mark the success/failed launches for each site on the map
import folium
import pandas as pd
from folium.features import DivIcon
from IPython.display import display, HTML

# Sample launch data with success/failure
launch_data = pd.DataFrame({
    'LaunchSite': [
        'CCAFS LC-40', 'CCAFS LC-40',
        'KSC LC-39A', 'KSC LC-39A',
        'VAFB SLC-4E'
    ],
    'Lat': [
        28.5618571, 28.5618571,
        28.6080585, 28.6080585,
        34.632834
    ],
    'Long': [
        -80.577366, -80.577366,
        -80.604444, -80.604444,
        -120.610746
    ],
    'Class': [1, 0, 1, 1, 0]  # 1 = Success, 0 = Failure
})

# NASA JSC as starting location
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add markers
for _, row in launch_data.iterrows():
    color = 'green' if row['Class'] == 1 else 'red'
    label = 'Success' if row['Class'] == 1 else 'Failure'
    coordinate = [row['Lat'], row['Long']]
    
    # Circle marker
    folium.Circle(
        coordinate,
        radius=500,
        color=color,
        fill=True,
        fill_color=color
    ).add_child(
        folium.Popup(f"{row['LaunchSite']} - {label}")
    ).add_to(site_map)
    
    # Text label
    folium.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size:12px; color:{color};"><b>{label}</b></div>'
        )
    ).add_to(site_map)

# Force render
display(HTML(site_map._repr_html_()))


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [15]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [16]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [17]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [18]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [20]:
import pandas as pd
import math

# Haversine distance function (in KM)
def haversine(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    R = 6371  # Earth radius in km
    
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1) * math.cos(phi2) * math.sin(dlambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c

# Launch site coordinates
launch_sites = pd.DataFrame({
    'LaunchSite': [
        'CCAFS LC-40',
        'KSC LC-39A',
        'VAFB SLC-4E'
    ],
    'Lat': [
        28.5618571,
        28.6080585,
        34.632834
    ],
    'Long': [
        -80.577366,
        -80.604444,
        -120.610746
    ]
})

# Approximate nearest coastline points
coastline_points = {
    'CCAFS LC-40': (28.563, -80.567),
    'KSC LC-39A': (28.609, -80.590),
    'VAFB SLC-4E': (34.632, -120.610)
}

# Calculate distances
distance_results = []

for _, site in launch_sites.iterrows():
    site_coord = (site['Lat'], site['Long'])
    coast_coord = coastline_points[site['LaunchSite']]
    
    distance_km = haversine(site_coord, coast_coord)
    
    distance_results.append({
        'LaunchSite': site['LaunchSite'],
        'Distance_to_Coast_km': round(distance_km, 2)
    })

# Result dataframe
distance_df = pd.DataFrame(distance_results)
distance_df


,LaunchSite,Distance_to_Coast_km
0,CCAFS LC-40,1.02
1,KSC LC-39A,1.41
2,VAFB SLC-4E,0.12


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [22]:
import folium
from IPython.display import display, HTML

# Create map
site_map = folium.Map(location=[29.559684888503615, -95.0830971930759], zoom_start=5)

# Add click-for-coordinate popup
site_map.add_child(
    folium.LatLngPopup()
)

# Force display
display(HTML(site_map._repr_html_()))


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [33]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance


*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [34]:
launch_site_lat = 28.5618571
launch_site_lon = -80.577366

coastline_lat = 28.56367
coastline_lon = -80.57163


In [35]:
launch_site_lat = 28.5618571
launch_site_lon = -80.577366

coastline_lat = 28.56367
coastline_lon = -80.57163


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [36]:
distance_coastline = calculate_distance(
    launch_site_lat,
    launch_site_lon,
    coastline_lat,
    coastline_lon
)

distance_coastline


0.5955418110769317

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [37]:
coordinates = [
    [launch_site_lat, launch_site_lon],
    [coastline_lat, coastline_lon]
]


In [38]:
lines = folium.PolyLine(
    locations=coordinates,
    weight=1,
    color='blue'
)


In [39]:
site_map.add_child(lines)
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size:12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline)
    )
)
site_map.add_child(distance_marker)
from IPython.display import display, HTML
display(HTML(site_map._repr_html_()))


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
